* YOLO拥有75个卷积层，还有跳过连接和上采样层。它不使用任何形式的池化，使用步幅为 2 的卷积层对特征图进行下采样。这有助于防止通常由池化导致的低级特征丢失。
* 卷积层所学习的特征会被传递到分类器/回归器，从而进行预测（边界框的坐标、类别标签等）
* 在 YOLO 中，预测是通过卷积层完成的其核心尺寸为：1×1×(B×(5+C))(分别描述每个边界框的中心坐标、维度、objectness 分数和 C 类置信度), 预测B个框每个框有5个参数和C个类别的预测概率
* 预测图就是每个可以预测固定数量边界框的单元格
* 如果对象的中心(其实就是ground truth方框的中心)位于单元格的感受野内，你会希望特征图的每个单元格都可以通过其中一个边界框预测对象
* ![预测对像的示意图](./2.jpg )
* 红色单元格是网格中第七行的第七个。我们现在使特征图中第七行第七个单元格（特征图中的对应单元格）作为检测狗的单元.现在，这个单元格可以预测三个边界框。哪个将会分配给狗的真值标签(预测边界框的宽度和高度看起来非常合理，但在实践中，训练会带来不稳定的梯度。所以，现在大部分目标检测器都是预测对数空间（log-space）变换，或者预测与预训练默认边界框（即锚点）之间的偏移)负责检测狗的边界框的锚点有最高的 IoU，且有真值框
* 下面公式描述了网络输出是如何转换的，以获得边界框的预测结果
* ![网络输出的转换公式](./3.jpg)
* 正常情况下，YOLO 不会预测边界框中心的确切坐标。它预测：与预测目标的网格单元左上角相关的偏移；使用特征图单元的维度(1)进行归一化的偏移。以我们的图像为例。如果中心的预测是 (0.4, 0.7)，则中心在 13 x 13 特征图上的坐标是 (6.4, 6.7)（红色单元的左上角坐标是 (6,6)）
* 但是，如果预测到的 x,y 坐标大于 1，比如 (1.2, 0.7)。那么中心坐标是 (7.2, 6.7)。注意该中心在红色单元右侧的单元中，或第 7 行的第 8 个单元。这打破了 YOLO 背后的理论，因为如果我们假设红色框负责预测目标狗，那么狗的中心必须在红色单元中，不应该在它旁边的网格单元中。因此，为了解决这个问题，我们对输出执行 sigmoid 函数，将输出压缩到区间 0 到 1 之间，有效确保中心处于执行预测的网格单元中
* 得出的预测 bw 和 bh 使用图像的高和宽进行归一化。即，如果包含目标（狗）的框的预测 bx 和 by 是 (0.3, 0.8)，那么 13 x 13 特征图的实际宽和高是 (13 x 0.3, 13 x 0.8)
* ![检测器输出在最终预测之前的变换过程](./4.jpg)
* Object 分数表示目标在边界框内的概率。红色网格和相邻网格的 Object 分数应该接近 1，而角落处的网格的 Object 分数可能接近 0。objectness 分数的计算也使用 sigmoid 函数，因此它可以被理解为概率。
* 类别置信度表示检测到的对象属于某个类别的概率（如狗、猫、香蕉、汽车等）。在 v3 之前，YOLO 需要对类别分数执行 softmax 函数操作。但是，YOLO v3 舍弃了这种设计，作者选择使用 sigmoid 函数。因为对类别分数执行 softmax 操作的前提是类别是互斥的。简言之，如果对象属于一个类别，那么必须确保其不属于另一个类别。这在我们设置检测器的 COCO 数据集上是正确的。但是，当出现类别「女性」（Women）和「人」（Person）时，该假设不可行。这就是作者选择不使用 Softmax 激活函数的原因
* YOLO v3 在 3 个不同尺度上进行预测。检测层用于在三个不同大小的特征图上执行预测，特征图步幅分别是 32、16、8。这意味着，当输入图像大小是 416 x 416 时，我们在尺度 13 x 13、26 x 26 和 52 x 52 上执行检测
* 该网络在第一个检测层之前对输入图像执行下采样，检测层使用步幅为 32 的层的特征图执行检测。随后在执行因子为 2 的上采样后，并与前一个层的特征图（特征图大小相同）拼接。另一个检测在步幅为 16 的层中执行。重复同样的上采样步骤，最后一个检测在步幅为 8 的层中执行。在每个尺度上，每个单元使用 3 个锚点预测 3 个边界框，锚点的总数为 9（不同尺度的锚点不同）。
![不同特征图上上采样预测](./5.jpg)
* 对于大小为 416 x 416 的图像，YOLO 预测 ((52 x 52) + (26 x 26) + 13 x 13)) x 3 = 10647 个边界框。但是，我们的示例中只有一个对象——一只狗。那么我们怎么才能将检测次数从 10647 减少到 1 呢？目标置信度阈值：首先，我们根据它们的 objectness 分数过滤边界框。通常，分数低于阈值的边界框会被忽略。非极大值抑制：非极大值抑制（NMS）可解决对同一个图像的多次检测的问题。例如，红色网格单元的 3 个边界框可以检测一个框，或者临近网格可检测相同对象。

In [1]:

from darknet import *
from util import predict_transform
import cv2
%load_ext autoreload
%autoreload 2


usage: ipykernel_launcher.py [-h] [--images IMAGES] [--det DET] [--bs BS]
                             [--confidence CONFIDENCE]
                             [--nms_thresh NMS_THRESH] [--cfg CFGFILE]
                             [--weights WEIGHTSFILE] [--reso RESO]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\King\AppData\Roaming\jupyter\runtime\kernel-737b3e1c-50b7-4880-8820-e9db0a3d0fd7.json


SystemExit: 2

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [55]:
#检测对配置文件的解析是否对
parse_cfg('.\cfg\yolov3.cfg')

[{'angle': '0',
  'batch': '1',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '416',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '1',
  'width': '416'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '2',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '1',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'linear', 'from': '-3', 'type': 'shortcut'},
 {'a

In [9]:
#检测创建网络
from darknet import *
blocks = parse_cfg('cfg/yolov3.cfg')
print(create_modules(blocks))

({'type': 'net', 'batch': '1', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (batch_nor

In [64]:

blocks = parse_cfg('cfg/yolov3.cfg')
print(create_modules(blocks))

[autoreload of darknet failed: Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 369, in superreload
    module = reload(module)
  File "D:\Anaconda\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "D:\Anaconda\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 608, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
  File "E:\ComputerVision\YOLO\darknet.py", line 132
    elif (x["type"] == "yolo"):
       ^
SyntaxError: invalid synt

({'type': 'net', 'batch': '1', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (batch_nor

[autoreload of darknet failed: Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 369, in superreload
    module = reload(module)
  File "D:\Anaconda\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "D:\Anaconda\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 608, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
  File "E:\ComputerVision\YOLO\darknet.py", line 135
    elif (x["type"] == "yolo"):
       ^
SyntaxError: invalid synt

NameError: name 'filters' is not defined

In [56]:
#测试反向传播
'''
对于批量中的图像，我们会有一个 100647×85 的表，它的每一行表示一个边界框（4 个边界框属性、1 个 objectness 分数和 80 个类别分数）
'''
model = Darknet("cfg/yolov3.cfg")
inp = get_test_input()
pred = model(inp,False)
print (pred)


tensor([[[  15.7473,   16.0189,  110.7126,  ...,    0.5069,
             0.4354,    0.5263],
         [  15.6137,   16.1746,  173.0657,  ...,    0.5046,
             0.4697,    0.5124],
         [  17.8154,   14.8297,  364.2511,  ...,    0.5793,
             0.4500,    0.4687],
         ...,
         [ 412.6372,  412.0495,    9.1626,  ...,    0.5292,
             0.4656,    0.4692],
         [ 411.9863,  411.6248,   19.2783,  ...,    0.4350,
             0.4629,    0.4556],
         [ 412.3563,  411.9576,   25.9322,  ...,    0.4133,
             0.5264,    0.4510]]])


In [1]:
%run detector.py --images dog-cycle-car.png --det det

Loading network.....
Network successfully loaded
E:\ComputerVision\YOLO\dog-cycle-car.png predicted in  3.333 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
SUMMARY
----------------------------------------------------------
Task                     : Time Taken (in seconds)

Reading addresses        : 0.000
Loading batch            : 0.020
Detection (1 images)     : 3.336
Output Processing        : 0.000
Drawing Boxes            : 1.360
Average time_per_img     : 4.716
----------------------------------------------------------


In [2]:
import torch

In [36]:
prediction = torch.FloatTensor([[1,2],[4,5]])

In [53]:
prediction[:,-1]

IndexError: too many indices for tensor of dimension 1